In [3]:
def get_met_objects():
    url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects'
    object_id_list = []
    import requests
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                return list()
    except:
        print("Issue running the requests function")
        return list()
    
    object_id_list = response_data['objectIDs']
    url_list = []
    for x in object_id_list:
        url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'+str(x)
        url_list.append(url)
        
    return url_list

In [4]:
url_list = get_met_objects()
import grequests
import time
start_time = time.time()
# try:


requests = (grequests.get(url) for url in url_list)
responses = grequests.map(requests)
print("dict: --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
json = [response.json() for response in responses]
pprint.pprint(json)
print("dict: --- %s seconds ---" % (time.time() - start_time))

#     if not response.status_code == 200:
#         print("HTTP error",response.status_code)
#     else:
#         try:
#             response_data = response.json()
#         except:
#             return dict()
# except:
#     print("Issue running the requests function")
#     return dict()

ModuleNotFoundError: No module named 'grequests'

In [ ]:
def get_object_info(url_list):  
    
    import time
    start_time = time.time()

    import requests

    try:
        response = (grequests.get(url) for url in url_list)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                return dict()
    except:
        print("Issue running the requests function")
        return dict()
    
    print("get json: --- %s seconds ---" % (time.time() - start_time))
    
    import time
    start_time = time.time()
    
    info_dict = dict()
    for x in response_data.items():
        if x[0] == 'objectID':
            continue
        info_dict[x[0]] = x[1]
        
    print("info dict: --- %s seconds ---" % (time.time() - start_time))
    
    return info_dict

In [ ]:
def art_database():
    import time
    object_id_list = range(1,50)

#     start_time = time.time()
#     object_id_list = get_met_objects()
#     print("get json: --- %s seconds ---" % (time.time() - start_time))
    
    start_time = time.time()
    database_dict = dict.fromkeys(object_id_list,None)
    print("dict: --- %s seconds ---" % (time.time() - start_time))
    
    url_list = []
    for x in database_dict.keys():
        url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'+str(x)
        url_list.append(url)
        
        
        info_dict = get_object_info(url_list)
        database_dict[x] = info_dict
    
    return database_dict

In [ ]:
def art_database_2():
    import time
    object_id_list = range(1,50)

#     start_time = time.time()
#     object_id_list = get_met_objects()
#     print("get json: --- %s seconds ---" % (time.time() - start_time))
    
    start_time = time.time()
    database_dict = dict.fromkeys(object_id_list,None)
    print("dict: --- %s seconds ---" % (time.time() - start_time))
    
    for x in database_dict.keys():
        url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'+str(x)
        info_dict = get_object_info(url)
        database_dict[x] = info_dict
    
    return database_dict

In [ ]:
question_pool = {
    'Which era is this art from': 'era'
    'Who created this art': 'artistDisplayName'
    'What is the name of this art': 'title'
}

In [ ]:
art_database()